# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [10]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [36]:
df_qty = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
df_qty

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [31]:
df_pivot = pd.pivot_table(df_qty, columns='CustomerID', index='ProductName', values='Quantity', aggfunc={'Quantity':'sum'}, fill_value=0)
df_pivot_tr = df_pivot.transpose()
df_pivot_tr

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [32]:
pdist(df_pivot_tr, metric='euclidean')

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [33]:
squareform(pdist(df_pivot_tr, metric='euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [38]:
dist_eucl = 1 / (1 + squareform(pdist(df_pivot_tr, metric='euclidean')))
distances = pd.DataFrame(dist_eucl, index=df_pivot_tr.index, columns=df_pivot_tr.index)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [49]:
similarities = distances[200].sort_values(ascending=False)[1:6]
sim_list = list(similarities.index)
sim_list

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [60]:
df_pivot_top_5 = pd.DataFrame(df_pivot, columns=sim_list)
df_pivot_top_5

,3317,1072,3535,1920,3909
ProductName,,,,,
Anchovy Paste - 56 G Tube,0,0,1,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,1,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0
Appetizer - Sausage Rolls,0,1,0,0,0
Apricots - Dried,0,0,1,0,0
...,...,...,...,...,...
Yeast Dry - Fermipan,0,1,0,0,0
Yoghurt Tubes,0,0,1,0,0
"Yogurt - Blueberry, 175 Gr",0,0,0,0,0


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [181]:
df_pivot_top_5['Total'] = df_pivot_top_5.sum(axis=1)
df_pivot_top_5.sort_values('Total', ascending=False).head(8)

,3317,1072,3535,1920,3909,Total
ProductName,,,,,,
Soup - Campbells Bean Medley,1,1,1,0,1,20
Muffin - Carrot Individual Wrap,0,2,0,1,0,15
Bay Leaf,1,0,0,0,2,15
Lamb - Ground,1,0,1,1,0,15
"Pepper - Black, Whole",2,0,1,0,0,15
Wanton Wrap,0,0,1,1,1,15
Pork - Kidney,0,0,0,3,0,15
"Cheese - Boursin, Garlic / Herbs",0,0,0,2,0,10


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [74]:
df_cust_200 = pd.DataFrame(df_pivot[200])
df_cust_200


,200
ProductName,
Anchovy Paste - 56 G Tube,0
"Appetizer - Mini Egg Roll, Shrimp",0
Appetizer - Mushroom Tart,0
Appetizer - Sausage Rolls,0
Apricots - Dried,0
...,...
Yeast Dry - Fermipan,0
Yoghurt Tubes,0
"Yogurt - Blueberry, 175 Gr",1


In [163]:
df_concat = pd.concat([df_cust_200, df_pivot_top_5], axis=1).sort_values('Total', ascending=False)
recomm = df_concat[df_concat[200] == 0].sort_values('Total', ascending=False).head(5).index
list_recomm = list(recomm)
list_recomm

['Soup - Campbells Bean Medley',
 'Bay Leaf',
 'Wanton Wrap',
 'Pork - Kidney',
 'Muffin - Carrot Individual Wrap']

In [164]:
df_concat[df_concat[200] == 0].sort_values('Total', ascending=False).head(5).index

Index(['Soup - Campbells Bean Medley', 'Bay Leaf', 'Wanton Wrap',
       'Pork - Kidney', 'Muffin - Carrot Individual Wrap'],
      dtype='object', name='ProductName')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [165]:
customerids = list(df_pivot_tr.index)
customerids

[33,
 200,
 264,
 356,
 412,
 464,
 477,
 639,
 649,
 669,
 694,
 756,
 883,
 891,
 1008,
 1034,
 1066,
 1072,
 1336,
 1428,
 1435,
 1534,
 1577,
 1594,
 1754,
 1839,
 1920,
 2187,
 2329,
 2503,
 2556,
 2566,
 2582,
 2617,
 2686,
 2754,
 2776,
 2902,
 2915,
 2939,
 3074,
 3253,
 3267,
 3305,
 3317,
 3472,
 3531,
 3535,
 3544,
 3885,
 3903,
 3909,
 4261,
 4479,
 4595,
 4644,
 4723,
 5104,
 5177,
 5183,
 5224,
 5230,
 5434,
 5678,
 5739,
 5832,
 5921,
 5968,
 5986,
 5997,
 6001,
 6185,
 6317,
 6352,
 6367,
 6447,
 6543,
 6734,
 6817,
 6851,
 7021,
 7134,
 7223,
 7543,
 7606,
 7623,
 7772,
 7827,
 7987,
 7999,
 8316,
 8367,
 8372,
 8539,
 8711,
 8917,
 8962,
 9047,
 9132,
 9260,
 9302,
 9533,
 9546,
 9597,
 9771,
 9804,
 10016,
 10094,
 10114,
 10224,
 10311,
 10418,
 10739,
 11023,
 11057,
 11083,
 11102,
 11253,
 11325,
 11380,
 11441,
 11443,
 11614,
 11628,
 11744,
 11883,
 11933,
 12059,
 12206,
 12232,
 12381,
 12529,
 12561,
 12596,
 12598,
 12852,
 12888,
 13062,
 13118,
 13338,
 

In [191]:
recomm_dict = dict()

In [201]:
def recommendations(customerid):
    '''
    Function that generates the recomendation for each customer ID
    
    '''
    dist_eucl = 1 / (1 + squareform(pdist(df_pivot_tr, metric='euclidean')))
    distances = pd.DataFrame(dist_eucl, index=df_pivot_tr.index, columns=df_pivot_tr.index)
    similarities = distances[customerid].sort_values(ascending=False)[1:6]
    sim_list = list(similarities.index)
    df_pivot_top_5 = pd.DataFrame(df_pivot, columns=sim_list)
    df_pivot_top_5['Total'] = df_pivot_top_5.sum(axis=1)
    df_cust_id = pd.DataFrame(df_pivot[customerid])
    df_concat = pd.concat([df_cust_id, df_pivot_top_5], axis=1).sort_values('Total', ascending=False)
    recommendation = list(df_concat[df_concat[customerid] == 0].sort_values('Total', ascending=False).head(5).index)
    recomm_dict[customerid] = recommendation 
    return recomm_dict
    

In [189]:
data1 = recommendations(33)
data1
recomm_dict

{33: ['Soup - Campbells Bean Medley',
  'Wine - Ej Gallo Sierra Valley',
  'Butter - Unsalted',
  'Wine - Blue Nun Qualitatswein',
  'Wine - Wyndham Estate Bin 777']}

In [194]:
for i in customerids:
    recommendations(i)

In [195]:
recomm_dict

{33: ['Soup - Campbells Bean Medley',
  'Wine - Ej Gallo Sierra Valley',
  'Butter - Unsalted',
  'Wine - Blue Nun Qualitatswein',
  'Wine - Wyndham Estate Bin 777'],
 200: ['Soup - Campbells Bean Medley',
  'Bay Leaf',
  'Wanton Wrap',
  'Pork - Kidney',
  'Muffin - Carrot Individual Wrap'],
 264: ['Fish - Scallops, Cold Smoked',
  'Veal - Inside, Choice',
  'Fondant - Icing',
  'Wine - Two Oceans Cabernet',
  'Soupfoamcont12oz 112con'],
 356: ['Butter - Unsalted',
  'Veal - Inside, Choice',
  'Nut - Chestnuts, Whole',
  'Beets - Candy Cane, Organic',
  'Foam Cup 6 Oz'],
 412: ['Soup - Campbells Bean Medley',
  'Cheese - Cambozola',
  'Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein',
  'Sauce - Gravy, Au Jus, Mix'],
 464: ['Butter - Unsalted',
  'Peas - Pigeon, Dry',
  'Pepper - Black, Whole',
  'Sausage - Breakfast',
  'Sauce - Gravy, Au Jus, Mix'],
 477: ['Olive - Spread Tapenade',
  'Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein',
  'Veal - Inside, Choice',
  '

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [197]:
pd.DataFrame.from_dict(recomm_dict)

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Soup - Campbells Bean Medley,Soup - Campbells Bean Medley,"Fish - Scallops, Cold Smoked",Butter - Unsalted,Soup - Campbells Bean Medley,Butter - Unsalted,Olive - Spread Tapenade,Wine - Ej Gallo Sierra Valley,Lamb - Ground,"Veal - Inside, Choice",...,Cheese - Wine,Muffin Batt - Choc Chk,"Pork - Loin, Bone - In",Grenadine,"Placemat - Scallop, White","Soup - Campbells, Lentil",Skirt - 29 Foot,Bread - Raisin Walnut Oval,Pernod,Beans - Wax
1,Wine - Ej Gallo Sierra Valley,Bay Leaf,"Veal - Inside, Choice","Veal - Inside, Choice",Cheese - Cambozola,"Peas - Pigeon, Dry","Pepper - Black, Whole",Cod - Black Whole Fillet,Wine - Magnotta - Belpaese,Sardines,...,"Lentils - Red, Dry",Papayas,Potatoes - Idaho 100 Count,Fenngreek Seed,Rum - Mount Gay Eclipes,"Cheese - Brie,danish",Beans - Kidney White,"Wine - Red, Harrow Estates, Cab",Crackers - Trio,Sherry - Dry
2,Butter - Unsalted,Wanton Wrap,Fondant - Icing,"Nut - Chestnuts, Whole","Pepper - Black, Whole","Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Soupfoamcont12oz 112con,Bar - Granola Trail Mix Fruit Nut,Wine - Blue Nun Qualitatswein,...,Meldea Green Tea Liquor,Meldea Green Tea Liquor,Pants Custom Dry Clean,"Wine - Red, Colio Cabernet",Tofu - Firm,Wiberg Super Cure,Cheese - Taleggio D.o.p.,Tea - English Breakfast,Tea - Jasmin Green,Pork - Hock And Feet Attached
3,Wine - Blue Nun Qualitatswein,Pork - Kidney,Wine - Two Oceans Cabernet,"Beets - Candy Cane, Organic",Wine - Blue Nun Qualitatswein,Sausage - Breakfast,"Veal - Inside, Choice","Wine - Magnotta, Merlot Sr Vqa","Peas - Pigeon, Dry",Wine - Ej Gallo Sierra Valley,...,Muffin - Carrot Individual Wrap,Mussels - Frozen,Cake - Box Window 10x10x2.5,Halibut - Fletches,"Cheese - Boursin, Garlic / Herbs",Pants Custom Dry Clean,Milk - 1%,Chips Potato Salt Vinegar 43g,"Peas - Pigeon, Dry",Pears - Bosc
4,Wine - Wyndham Estate Bin 777,Muffin - Carrot Individual Wrap,Soupfoamcont12oz 112con,Foam Cup 6 Oz,"Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,"Pepper - Black, Whole",Appetizer - Mushroom Tart,"Pepper - Black, Whole",...,Coffee - Dark Roast,Wine - Chablis 2003 Champs,Spice - Peppercorn Melange,Ice Cream Bar - Oreo Cone,Sobe - Tropical Energy,Shrimp - 31/40,Yoghurt Tubes,Halibut - Steaks,Pastry - Choclate Baked,Ecolab - Mikroklene 4/4 L


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [198]:
customerids = list(df_pivot_tr.index)
customerids

[33,
 200,
 264,
 356,
 412,
 464,
 477,
 639,
 649,
 669,
 694,
 756,
 883,
 891,
 1008,
 1034,
 1066,
 1072,
 1336,
 1428,
 1435,
 1534,
 1577,
 1594,
 1754,
 1839,
 1920,
 2187,
 2329,
 2503,
 2556,
 2566,
 2582,
 2617,
 2686,
 2754,
 2776,
 2902,
 2915,
 2939,
 3074,
 3253,
 3267,
 3305,
 3317,
 3472,
 3531,
 3535,
 3544,
 3885,
 3903,
 3909,
 4261,
 4479,
 4595,
 4644,
 4723,
 5104,
 5177,
 5183,
 5224,
 5230,
 5434,
 5678,
 5739,
 5832,
 5921,
 5968,
 5986,
 5997,
 6001,
 6185,
 6317,
 6352,
 6367,
 6447,
 6543,
 6734,
 6817,
 6851,
 7021,
 7134,
 7223,
 7543,
 7606,
 7623,
 7772,
 7827,
 7987,
 7999,
 8316,
 8367,
 8372,
 8539,
 8711,
 8917,
 8962,
 9047,
 9132,
 9260,
 9302,
 9533,
 9546,
 9597,
 9771,
 9804,
 10016,
 10094,
 10114,
 10224,
 10311,
 10418,
 10739,
 11023,
 11057,
 11083,
 11102,
 11253,
 11325,
 11380,
 11441,
 11443,
 11614,
 11628,
 11744,
 11883,
 11933,
 12059,
 12206,
 12232,
 12381,
 12529,
 12561,
 12596,
 12598,
 12852,
 12888,
 13062,
 13118,
 13338,
 

In [199]:
recomm_dict_cityblock = dict()

In [202]:
def recommendations_cityblock(customerid):
    '''
    Function that generates the recomendation for each customer ID
    
    '''
    dist_eucl = 1 / (1 + squareform(pdist(df_pivot_tr, metric='cityblock')))
    distances = pd.DataFrame(dist_eucl, index=df_pivot_tr.index, columns=df_pivot_tr.index)
    similarities = distances[customerid].sort_values(ascending=False)[1:6]
    sim_list = list(similarities.index)
    df_pivot_top_5 = pd.DataFrame(df_pivot, columns=sim_list)
    df_pivot_top_5['Total'] = df_pivot_top_5.sum(axis=1)
    df_cust_id = pd.DataFrame(df_pivot[customerid])
    df_concat = pd.concat([df_cust_id, df_pivot_top_5], axis=1).sort_values('Total', ascending=False)
    recommendation = list(df_concat[df_concat[customerid] == 0].sort_values('Total', ascending=False).head(5).index)
    recomm_dict_cityblock[customerid] = recommendation 
    return recomm_dict_cityblock

In [203]:
for i in customerids:
    recommendations_cityblock(i)


In [204]:
final_df_cityblock = pd.DataFrame.from_dict(recomm_dict_cityblock)
final_df_cityblock

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,"Oranges - Navel, 72",Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Cheese - Mozzarella,Olive - Spread Tapenade,"Cheese - Boursin, Garlic / Herbs",Ice Cream Bar - Hageen Daz To,Ice Cream Bar - Hageen Daz To,...,Bar - Granola Trail Mix Fruit Nut,Butter - Unsalted,Cup - Translucent 7 Oz Clear,"Sole - Dover, Whole, Fresh",Wine - Blue Nun Qualitatswein,Wine - Chardonnay South,Veal - Sweetbread,Sherry - Dry,"Wine - White, Colubia Cresh",Wine - Blue Nun Qualitatswein
1,Wine - Blue Nun Qualitatswein,Pomello,Bread - Italian Roll With Herbs,"Veal - Inside, Choice",Table Cloth 81x81 White,Knife Plastic - White,"Pepper - Black, Whole",Pork - Kidney,Lamb - Ground,Lamb - Ground,...,Wine - Blue Nun Qualitatswein,Juice - Lime,Knife Plastic - White,"Beef - Chuck, Boneless",Pork - Kidney,Butter - Unsalted,Bar - Granola Trail Mix Fruit Nut,Soup - Campbells Bean Medley,Ice Cream Bar - Hageen Daz To,Jagermeister
2,Soup - Campbells Bean Medley,Scampi Tail,"Veal - Inside, Choice","Nut - Chestnuts, Whole",Butter - Unsalted,Pasta - Orecchiette,Wine - Blue Nun Qualitatswein,"Beets - Candy Cane, Organic","Pepper - Black, Whole","Pepper - Black, Whole",...,Beef Ground Medium,Pasta - Angel Hair,Muffin - Zero Transfat,Cornflakes,Sherry - Dry,Raspberries - Fresh,Butter - Unsalted,Bandage - Fexible 1x3,Pork - Kidney,Flavouring - Orange
3,Cheese Cloth No 100,Soup - Campbells Bean Medley,Pork - Kidney,"Beets - Candy Cane, Organic","Pepper - Black, Whole","Peas - Pigeon, Dry","Veal - Inside, Choice",Wine - Crozes Hermitage E.,Butter - Unsalted,Butter - Unsalted,...,Lettuce - Treviso,Bay Leaf,Wine - Crozes Hermitage E.,"Pepper - Black, Whole",Lamb - Ground,Wine - Gato Negro Cabernet,Beef - Montreal Smoked Brisket,"Wine - Magnotta, Merlot Sr Vqa",Hersey Shakes,Tray - 16in Rnd Blk
4,Remy Red,Pork - Kidney,Wine - Two Oceans Cabernet,Foam Cup 6 Oz,Soup - Campbells Bean Medley,Butter - Unsalted,Sprouts - Baby Pea Tendrils,"Pasta - Detalini, White, Fresh",Pork - Kidney,Wine - Ej Gallo Sierra Valley,...,Knife Plastic - White,Bread Crumbs - Panko,Pork - Inside,Watercress,Table Cloth 81x81 White,Tea - Decaf Lipton,Tia Maria,Butter - Unsalted,Lime Cordial - Roses,"Wine - Magnotta, Merlot Sr Vqa"
